# `CLOUD.jl` - 2D linear advection example

In [10]:
using CLOUD, OrdinaryDiffEq

Define the physical problem

In [11]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(cos(θ)))) # end time for one period

conservation_law = LinearAdvectionEquation((a*cos(θ),a*sin(θ)))
initial_data = InitialDataSine(A,k)
exact_solution = ExactSolution(conservation_law,initial_data);

Set discretization parameters

In [12]:
M = 4
p = 5
p_map = 5
form = WeakConservationForm()
strategy = Lazy()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.1*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

## Collocated DGSEM on quadrilaterals

Set up a `DGSEM` scheme with LG quadrature on quadrilaterals of degree `p` with `M` edges in each direction

In [13]:
reference_approximation =ReferenceApproximation(
    DGSEM(p), Quad(),
    volume_quadrature_rule=LGQuadrature(),
    facet_quadrature_rule=LGQuadrature(),
    mapping_degree=p_map, N_plot=50)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.2)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

"results/advection_2d_dgsem/"

Run the solver

In [14]:
ode_problem_1 = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy)

save_solution(ode_problem_1.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem_1, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:          105μs / 986610.4%        10.6KiB / 9432835.1%  

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual          49.9k    734ms   71.2%  14.7μs    466MiB   47.8%  9.56KiB
   volume terms         49.9k    460ms   44.6%  9.23μs    259MiB   26.7%  5.33KiB
   mass matrix solve    49.9k    100ms    9.6%  2.00μs   35.0MiB    3.6%     736B
   facet terms          49.9k   97.8ms    9.5%  1.96μs    123MiB   12.7%  2.53KiB
 extrap solution        49.9k    107ms   10.3%  2.14μs   95.1MiB    9.8%  1.95KiB
 eval num flux          49.9k   90.3ms    8.8%  1.81μs    148MiB   15.2%  3.03KiB
 gather ext sta

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [15]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data, T)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.0011999302847437096
Conservation (initial/final/diff):
[-1.3247403066879437e-8][-1.3247403068614161e-8][-1.734723475976807e-18]
Energy (initial/final/diff):
[0.12499988821253886][0.12494939330671882][-5.0494905820042124e-5]


## Modal DG scheme on triangles

Set up a `DGMulti` scheme on triangles of degree `p` with `M` edges in each direction

In [16]:
reference_approximation = ReferenceApproximation(
    DGMulti(p), Tri(), mapping_degree=p_map, N_plot=10)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.1)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgmulti/", overwrite=true, clear=true)

"results/advection_2d_dgmulti/"

Run the solver and print timer outputs

In [17]:
ode_problem_2 = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy)

save_solution(ode_problem_2.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem_2, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:         78.5μs / 6344183.9%      10.6KiB / 16579353.4%  

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual           100k    4.03s   80.9%  40.4μs    806MiB   47.2%  8.28KiB
   mass matrix solve     100k    3.31s   66.5%  33.2μs    396MiB   23.1%  4.06KiB
   volume terms          100k    433ms    8.7%  4.35μs    323MiB   18.9%  3.31KiB
   facet terms           100k   72.6ms    1.5%   728ns   19.8MiB    1.2%     208B
 eval flux               100k    358ms    7.2%  3.59μs    108MiB    6.3%  1.11KiB
 gather ext state        100k    214ms    4.3%  2.15μs    510MiB   29.8%  5.23KiB
 eval num flux 

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [18]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), exact_solution, T)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.00048784850808591414
Conservation (initial/final/diff):
[-6.5217300401244904e-9][-6.521728651044667e-9][1.3890798233884283e-15]
Energy (initial/final/diff):
[0.12499978051190094][0.12498968050064548][-1.0100011255467778e-5]
